# 강력한 정규 표현식의 세계로
---


## 메타문자
---
앞에서 살펴본 `+`, `*`, `[]`, `{}` 등의 메타문자는 매치가 진행될 때 현재 매치되고 있는 문자열의 위치가 변경된다(보통 소비된다고 표현). 하지만 이와 달리 문자열을 소비시키지 않는 메타 문자도 있다. 이번에는 이런 문자열 소비가 없는(zerowidth assertions) 메타 문자에 대해 살펴보자.

### 1. `|`
`|` 메타 문자는 or과 동일한 의미로 사용된다. `A|B` 라는 정규식이 있다면 A 또는 B라는 의미가 된다.

In [1]:
import re
p = re.compile('Crow|Servo')
m = p.match('CrowHello')
print(m)

<re.Match object; span=(0, 4), match='Crow'>


### 2. `^`
`^` 메타 문자는 문자열의 맨 처음과 일치함을 의미한다. 앞에서 살펴 본 컴파일 옵션 `re.MULTILINE` 을 사용할 경우에는 여러 줄의 문자열일 때 각 줄의 처음과 일치하게 된다.

In [2]:
print(re.search('^Life', 'Life is too short'))
print(re.search('^Life', 'My Life'))

<re.Match object; span=(0, 4), match='Life'>
None


`^Life` 정규식은 Life 문자열이 처음에 온 경우에는 매치하지만 처음 위치가 아닌 경우에는 매치되지 않는 것을 확인할 수 있다.

### 3. `$`
`$` 메타 문자는 `^` 메타 문자와 반대의 경우이다. 즉 `$`는 문자열의 끝과 매치함을 의미한다.

In [3]:
print(re.search('short$', 'Life is too short'))
print(re.search('short$', 'Life is too short, you need python'))

<re.Match object; span=(12, 17), match='short'>
None


`short$` 정규식은 검색할 문자열이 short로 끝난 경우에는 매치되지만 그 이외의 경우에는 매치되지 않는다.
> ※ `^` 또는 `$` 문자를 메타 문자가 아닌 문자 그 자체로 매치하고 싶은 경우에는 `\^`, `\$`로 사용하면 된다.

### 4. `\A`
`\A`는 문자열의 처음과 매치됨을 의미한다. `^`메타 문자와 동일한 의미이지만 `re.MULTILINE` 옵션을 사용할 경우에는 다르게 해석된다. `re.MULTILINE` 옵션을 사용할 경우 `^`은 각 줄의 문자열의 처음과 매치되지만 `\A`는 줄과 상관없이 전체 문자열의 처음하고만 매치된다.

In [7]:
# ^ 메타문자를 사용한 경우

import re
p = re.compile("^python\s\w+", re.MULTILINE)

data = """python one
life is too short
python two
you need python
python three
"""

print(p.findall(data))

['python one', 'python two', 'python three']


In [8]:
# \A 메타문자를 사용한 경우

import re
p = re.compile("\Apython\s\w+", re.MULTILINE)

data = """python one
life is too short
python two
you need python
python three
"""

print(p.findall(data))

['python one']


### 5. `\Z`
`\Z` 는 문자열의 끝과 매치됨을 의미한다. 이것 역시 `\A`와 동일하게 `re.MULTILINE` 옵션을 사용할 경우 `$` 메타 문자와는 달리 전체 문자열의 끝과 매치된다.

### 6. `\b`
`\b`는 단어 구분자(Word boundary)이다. 보통 단어는 whitespace에 의해 구분된다.

In [9]:
import re
p = re.compile(r'\bclass\b')
print(p.search('no class at all'))

<re.Match object; span=(3, 8), match='class'>


`\bclass\b` 정규식은 앞뒤가 whitespace로 구분된 class라는 단어와 매치됨을 의미한다. 따라서 no class at all의 class라는 단어와 매치됨을 확인할 수 있다.

In [10]:
print(p.search('the declassified algorithm'))

None


위의 예시는 the declassified algorithm 문자열 안에 class 문자열이 포함되어 있지만 whitespace로 구분된 단어가 아니기 때문에 매치되지 않는다.

In [11]:
print(p.search('one subclass is'))

None


위 예시도 마찬가지로 class 앞에 sub문자열이 붙어 있기 때문에 매치되지 않는다.  

`\b` 메타 문자를 사용할 때 주의해야할 점이 있는데 `\b`는 파이썬 리터럴 규칙에 의하면 백스페이스(Backspace)를 의미한다. 그래서 백스페이스가 아닌 단어 구분자임을 알려주기 위해 반드시 `r'\bclass\b'`처럼 Raw String임을 알려주는 기호 r을 붙여야 한다.

### 7. `\B`
`\B` 메타 문자는 `\b` 메타 문자와 반대의 경우이다. 즉 whitespace로 구분된 단어가 아닌 경우에만 매치된다.

In [12]:
import re
p = re.compile(r'\Bclass\B')
print(p.search('no class at all'))
print(p.search('the declassified algorithm'))
print(p.search('one subclass is'))

None
<re.Match object; span=(6, 11), match='class'>
None


위의 예시를 보면 class 단어 앞뒤에 whitespace가 하나라도 있는 경우에는 매치가 안되는 것을 확인할 수 있다.

## 그루핑(Grouping)
---
ABC 문자열이 계속해서 반복되는지 조사하는 정규식을 작성한다고 가정한다. 이 때 필요한 것이 그루핑(Grouping)이다. 그루핑을 사용해서 정규식을 작성하면 다음과 같다.
> (ABC)+

그룹을 만들어 주는 메타 문자는 바로 `( )`이다.

In [14]:
import re
p = re.compile('(ABC)+')
m = p.search('ABCABCABC OK?')
print(m)
print(m.group())

<re.Match object; span=(0, 9), match='ABCABCABC'>
ABCABCABC


다음 예시를 확인해보자.

In [15]:
p = re.compile(r"\w+\s+\d+[-]\d+[-]\d+")
m = p.search("park 010-1234-1234")

`\w+\s+\d+[-]\d+[-]\d+`은 `이름 + " " + 전화번호` 형태의 문자열을 찾는 정규식이다. 그런데 이렇게 매치된 문자열 중에서 이름만 뽑아내고 싶다면 어떻게 해야 할까?  

보통 반복되는 문자열을 찾을 때 그룹을 사용한다. 그룹을 사용하는 가장 큰 이유는 매치된 문자열 중에서 특정 부분의 문자열만 뽑아내기 위해서인 경우가 많다.  

위의 예시에서 '이름' 부분만 뽑아내고 싶다면 다음처럼 작성하면 된다.

In [20]:
p = re.compile(r"(\w+)\s+\d+[-]\d+[-]\d+")
m = p.search("park 010-1234-1234")
print(m.group(1))

park


이름에 해당하는 `\w+` 부분을 그룹 `(\w+)`으로 만들면 match 객체의 group(인덱스) 메서드를 사용하여 그루핑된 부분의 문자열만 뽑아낼 수 있다. group 메서드의 인덱스는 다음과 같은 의미를 가진다.

| group(인덱스) | 설명 |
| :--- | :--- |
| group(0) | 매치된 전체 문자열 |
| group(1) | 첫 번째 그룹에 해당하는 문자열 |
| group(2) | 두 번째 그룹에 해당하는 문자열 |
| group(n) | n 번째 그룹에 해당하는 문자열 |

In [21]:
import re
p = re.compile(r"(\w+)\s+(\d+[-]\d+[-]\d+)")
m = p.search("park 010-1234-1234")
print(m.group(0))
print(m.group(1))
print(m.group(2))

park 010-1234-1234
park
010-1234-1234


위처럼 전화번호 부분을 그룹 `(\d+[-]\d+[-]\d+)`로 만들었다. 이렇게 하면 `group(2)`처럼 사용해서 전화번호만 뽑아낼 수 있다.  
만약 현재 상태에서 전화번호의 국번에 해당하는 부분만 따로 뽑아내고 싶다면 한 번 더 그루핑하면 된다.

In [24]:
p = re.compile(r"(\w+)\s+((\d+)[-]\d+[-]\d+)")
m = p.search("park 042-1234-1234")
print(m.group(3))

042


이처럼 그루핑을 할 때는 그룹을 중첩되게 사용하는 것도 가능하다. 그룹이 중첩되어 있는 경우는 바깥쪽부터 시작해서 안쪽으로 들어갈 수록 인덱스가 증가한다.

### 그루핑된 문자열 재참조하기
그룹의 또 하나 좋은 점은 한 번 그루핑한 문자열을 재참조(Backreferences)할 수 있다는 점이다. 

In [25]:
import re
p = re.compile(r'(\b\w+)\s+\1')
p.search('Paris in the the rain').group()

'the the'

정규식 `(\b\w+)\s+\1`은 `(그룹) + " " + 그룹과 동일한 단어`와 매치됨을 의미한다. 이렇게 정규식을 만들게 되면 2개의 동일한 단어를 연속적으로 사용해야만 매치된다. 이것을 가능하게 해주는 것이 바로 재참조 메타 문자인 `\1`이다. `\1`은 정규식의 그룹 중 첫 번째 그룹을 가리킨다.
> ※ 두 번째 그룹을 참조하려면 `\2`를 사용하면 된다.

### 그루핑된 문자열에 이름 붙이기

정규식 안에 그룹이 10개 이상만 되어도 혼란스러울 것이다. 게다가 정규식이 수정되면서 그룹이 추가, 삭제되면 그 그룹을 인덱스로 참조한 프로그램도 모두 변경해 주어야 하는 위험도 갖게 된다.  
그룹을 인덱스가 아닌 이름(Named Groups)으로 참조할 수 있다면 이러한 문제에서 벗어날 수 있다.  

따라서 정규식은 그룹을 만들 때 그룹 이름을 지정할 수 있게 했다. 방법은 다음과 같다.
> `(?P<name>\w+)\s+((\d+)[-]\d+[-]\d+)`

위의 정규식은 앞서 다룬 이름과 전화번호를 추출하는 정규식과 동일한 기능을 수행한다. 달라진 부분은 다음과 같다.
> (\w+) --> (?P<name>\w+)

여기서 사용한 `(?...)` 표현식은 정규 표현식의 확장 구문이다. 이 확장 구문을 사용하기 시작하면 가독성이 상당히 떨어지지만 강력함을 가지게 된다. 그룹에 이름을 지어 주고 싶다면 다음과 같은 확장 구문을 사용해야 한다.
> (?P<그룹명>...)

그룹에 이름을 지정하고 참조하는 예를 살펴보자.

In [26]:
import re

p = re.compile(r"(?P<name>\w+)\s+((\d+)[-]\d+[-]\d+)")
m = p.search("Kim 010-1234-1234")
print(m.group("name"))

Kim


이처럼 name이라는 그룹 이름으로 참조가 가능하다. 그룹 이름을 사용하면 정규식 안에서 재참조하는 것도 가능하다.

In [27]:
import re

p = re.compile(r'(?P<word>\b\w+)\s+(?P=word)')
p.search('Maybe if you got rid of that old yee yee ass haircut').group()

'yee yee'

위와 같이 재참조할 때는 `(?P=그룹이름)` 이라는 확장 구문을 사용한다.

## 전방 탐색(Lookahead Assertions)
---

In [28]:
import re

p = re.compile(".+:")
m = p.search("https://www.google.com/")
print(m.group())

https:


정규식 `.+:`과 일치하는 문자열로 https:를 돌려주었다. 만약 https: 라는 검색 결과에서 :을 제외하고 출력하려면 어떻게 해야할까? 만약 위 정규식보다 훨씬 복잡한 정규식이고 그루핑은 추가로 할 수 없다는 조건까지 더해진다면 어떻게 해야할까?  

이 때 사용할 수 있는 것이 바로 전방 탐색이다. 전방 탐색에는 긍정(Positive)과 부정(Negative)의 2종류가 있고 다음과 같이 표현한다.

* 긍정형 전방 탐색(`(?=...)`) : `...`에 해당되는 정규식과 매치되어야 하며 조건이 통과되어도 문자열이 소비되지 않는다.
* 부정형 전방 탐색(`(?!...)`) : `...`에 해당되는 정규식과 매치되지 않아야 하며 조건이 통과되어도 문자열이 소비되지 않는다.

### 1. 긍정형 전방 탐색
긍정형 전방 탐색을 사용하면 http:의 결과를 http로 바꿀 수 있다. 다음 예시를 확인해보자.

In [29]:
import re
p = re.compile(".+(?=:)")
m = p.search("https://www.google.com/")
print(m.group())

https


기존 정규식에서 `:`에 해당하는 부분에 긍정형 전방 탐색 기법을 적용해서 `(?=:)`으로 변경했다. 이렇게 되면 기존 정규식과 검색에서는 동일한 효과를 발휘하지만 `:`에 해당하는 문자열이 정규식 엔진에 의해 소비되지 않아(검색에는 포함되지만 검색 결과에서는 제외됨) 검색 결과에서는 `:`이 제거된 후 돌려주는 효과가 있다.

> `.*[.].*$`

위 정규식은 `파일이름 + . + 확장자`를 나타내는 정규식이다. 이 정규식은 foo.bar, autoexec.bat, sendmail.cf와 같은 형식의 파일과 매치될 것이다.  
위 정규식에 확장자가 bat인 파일은 제외해야 한다는 내용의 조건을 추가해 보자.

In [ ]:
.*[.]([^b]..|.[^a].|..[^t])$

위 정규식은 `|` 메타 문자를 사용해서 확장자의 첫 번째 문자가 b가 아니거나 두 번째 문자가 a가 아니거나 세 번째 문자가 t가 아닌 경우를 의미한다. 위 정규식에 의해서 foo.bar는 제외되지 않고 autoexec.bat은 제외되어 원하는 결과를 돌려준다. 하지만 이 정규식은 sendmail.cf처럼 확장자의 문자 개수가 2개인 케이스를 포함하지 못한다. 이러한 문제를 해결하기 위해서는 다음과 같이 변형을 해야 한다.

In [ ]:
.*[.]([^b].?.?|.[^a]?.?|..?[^t]?)$

### 2. 부정형 전방 탐색
위에서 다룬 긍정형 전방 탐색에 조건을 추가할수록 정규식은 점점 복잡해졌다. 이 때 필요한 것이 부정형 전방 탐색이다. 이 방법을 사용하면 좀 더 간단하게 처리가 가능해진다.

In [ ]:
.*[.](?!bat$).*$

위의 정규식은 긍정형 전방 탐색에서 만들었던 정규식과 동일하게 확장자가 bat이 아닌 경우에만 통과된다는 의미를 가지고 있다. bat 문자열이 있는지 조사하는 과정에서 문자열이 소비되지 않기 때문에 bat이 아니라고 판단되면 다음 정규식 매치가 진행된다.  

bat외에도 exe도 제외하라는 조건을 추가한다면 다음처럼 간단하게 exe만 추가해주면 된다.

In [ ]:
.*[.](?!bat$).*$

## 문자열 바꾸기
---
sub 메서드를 사용하면 정규식과 매치되는 부분을 다른 문자로 쉽게 바꿀 수 있다.

In [2]:
import re
p = re.compile('(blue|white|red)')
p.sub('colour', 'blue socks and red shoes')

'colour socks and colour shoes'

sub 메서드의 첫 번째 매개변수는 "바꿀 문자열(replacement)"이 되고, 두 번째 매개변수는 "대상 문자열"이 된다. 위 예에서 볼 수 있듯이 blue 또는 white 또는 red라는 문자열이 colour라는 문자열로 바뀌는 것을 확인할 수 있다.  

그런데 단 한 번만 바꾸고 싶은 경우에는? 바꾸기 횟수를 제어하려면 다음처럼 세 번째 매개변수로 count 값을 넘기면 된다.

In [3]:
p.sub('colour', 'blue socks and red shoes', count=1)

'colour socks and red shoes'

이렇게 세 번째 매개변수를 설정할 경우, 처음 일치하는 blue만 colour로 변경된 것을 확인할 수 있다.  

#### \[sub 메서드와 유사한 subn 메서드\]

subn 역시 sub와 동일한 기능을 하지만 반환 결과를 튜플로 돌려준다는 차이가 있다. 돌려준 튜플의 첫 번째 요소는 변경된 문자열이고, 두 번째 요소는 바꾸기 발생한 횟수이다.

In [4]:
import re
p = re.compile('(blue|white|red)')
p.subn('colour', 'blue socks and red shoes')

('colour socks and colour shoes', 2)